https://mp.weixin.qq.com/s/ikaQ-yS0xr1TVLv32ybbcA

In [ ]:
# 没有上下文件管理之前，只能依赖于异常处理结构
#!/usr/bin/env python3

import logging
logging.basicConfig(level=logging.INFO)

def main():
    """向 /tmp/a.txt 文件中写入 "hello world ."
    """
    file_object = None
    try:
        logging.info("open file in try block. ")
        # 打开文件
        file_object = open("/tmp/a.txt", 'w')
        file_object.write("hello world .\n")
        
    except Exception as err:
        pass
    finally:
        # 关闭文件回收句柄
        logging.info("close file in finally block .")
        if file_object is not None:
            file_object.close()
        
main()

In [ ]:
# 使用with
#!/usr/bin/env python3

import logging
logging.basicConfig(level=logging.INFO)

def main():
    """向 /tmp/a.txt 文件中写入 "hello world ."
    """
    with open("/tmp/a.txt",'w') as file_object:
        file_object.write("hello world .\n")
        
        
main()

### with 背后的原理

with 语句背后的原理就是上下文管理协议，这个协议约定了进入上下文之前要调用对象的 __enter__ 方法，在退出的时候要调用对象的 __exit__ 方法；我们可以在 __enter__ 方法里面实现 try 部分的逻辑，而在 __exit__ 方法里面实现 finally 的逻辑。

In [ ]:
# 编写支持with的代码
#!/usr/bin/env python3

import logging
logging.basicConfig(level=logging.INFO)

class MFile(object):
    file_path:str = None
    _file_object = None
    
    def __init__(self, file_path:str=None):
        self.file_path = file_path
    
    def __enter__(self):
        logging.info("__enter__ function called .")
        self._file_object = open(self.file_path)
    
    def __exit__(self, *args, **kwargs):
        logging.info("__exit__ function called .")
        if self._file_object is not None:
            self._file_object.close()

def main():
    with MFile("/tmp/a.txt") as f:
        pass

main()

In [ ]:
# 运行
python3 main.py
INFO:root:__enter__ function called .
INFO:root:__exit__ function called .

In [ ]:
# 利用生成器简化
#!/usr/bin/env python3

import logging
import contextlib
logging.basicConfig(level=logging.INFO)

@contextlib.contextmanager
def MFile(file_path:str=None):
    # __enter__
    logging.info("__enter__ function called .")
    file_object = open(file_path, 'w')
    yield file_object
    # __exit__
    logging.info("__exit__ function called .")
    file_object.close()

def main():
    with MFile("/tmp/a.txt") as f:
        f.write("just a test str . \n")
        

main()

In [ ]:
# 实践
import os
import contextlib
from threading import RLock

_user_sudo_lock = RLock()

@contextlib.contextmanager
def sudo(message="sudo"):
    """临时升级权限到 root .
    """
    # 对于权限这个临界区的访问要串行化
    with _user_sudo_lock as lk:
        # 得到当前进程的 euid
        old_euid = os.geteuid()
        # 提升权限到 root
        os.seteuid(0)
        yield message
        # 恢复到普通权限
        os.seteuid(old_euid)
